In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from adaptive_latents import proSVD, CenteringTransformer, Pipeline, NumpyTimedDataSource, AnimationManager
import datasets
import itertools
import tqdm.notebook as tqdm

rng = np.random.default_rng()


In [ ]:
d = datasets.Naumann24uDataset(datasets.Naumann24uDataset.sub_datasets[0])
d.sub_dataset

In [ ]:
%matplotlib inline
fig, ax = plt.subplots()
d.plot_colors(ax)


## Experiment structure

In [ ]:
neuron_index = 65

fig, ax = plt.subplots(figsize=(12, 4))
ax.plot(d.C[neuron_index,:])
for idx, (time, angle, _) in d.visual_stimuli.iterrows():
    plt.axvline(time, color=d.a2c(angle), alpha=.5)
for time in d.opto_stimulations['sample']:
    plt.axvline(time, color='k', alpha=.5)

In [ ]:
t1, t2 = 0, 500
vis_presentation_window = np.array([0, 20])

fig, ax = plt.subplots(figsize=(12, 4))
plt.plot(d.C[neuron_index,t1:t2])
for idx, sample, l_angle, r_angle in d.visual_stimuli.itertuples():
    stim_window = sample + vis_presentation_window
    if t1 <= stim_window[0] and t2 >= stim_window[1]:
        ax.axvspan(stim_window[0], stim_window[1], alpha=0.2, color=d.a2c(l_angle))


## proSVD stability

In [ ]:
pro = proSVD(k=4, log_level=1)
p = Pipeline([
    CenteringTransformer(),
    pro,
])

p.offline_run_on(d.neural_data);

In [ ]:
fig, ax = plt.subplots()

pro.plot_Q_stability(ax)

for time in d.opto_stimulations['sample']:
    plt.axvline(time, color='k')
plt.xlim([1800, 3200])
plt.ylim([0, 0.01]);


## Reactions to stim in latent space

In [ ]:
pro = proSVD(k=10)
p = Pipeline([
    CenteringTransformer(),
    pro,
])

s = d.neural_data.t < d.opto_stimulations['sample'].min()
first_part_data = NumpyTimedDataSource(d.neural_data.a[s], d.neural_data.t[s])
second_part_data = NumpyTimedDataSource(d.neural_data.a[~s], d.neural_data.t[~s])


first_part_streams = p.offline_run_on(first_part_data, convinient_return=False)

whole_offline_streams = p.offline_run_on(d.neural_data, fit=False, convinient_return=False)
offline_t = np.array([x.t for x in whole_offline_streams[0]])
offline_output = np.squeeze(whole_offline_streams[0])

second_part_streams = p.offline_run_on(second_part_data, convinient_return=False)
assert set(first_part_streams.keys()) == set(second_part_streams.keys())
united_streams = {k: first_part_streams[k] + second_part_streams[k] for k in first_part_streams.keys()}

l = [x for x in united_streams[0] if not np.isnan(x).any()]
online_t = np.array([x.t for x in l])
online_output = np.squeeze(l)


In [ ]:
k = 10

fig, axs = plt.subplots(ncols=2, nrows=2, sharex=True, sharey=True, figsize=(8,8))
axis_0, axis_1 = 0,1

for row, output, t in [ (0, online_output, online_t), (1, offline_output, offline_t)]:
    lineness = {0: 'offline', 1: 'online'}.get(row)

    s = t < d.opto_stimulations.loc[0, 'sample']
    axs[row,0].scatter(output[s,axis_0], output[s,axis_1], s=10, c=t[s], lw=0)
    axs[row,1].scatter(output[~s,axis_0], output[~s,axis_1], s=10, lw=0, color=np.array([1,1,1])*.8,)

    r = np.arange(5*k,5*k+5)
    assert np.std(d.opto_stimulations.loc[r, 'target_neuron']) == 0
    for i in r:
        s = (d.opto_stimulations.loc[i, 'sample'] <= t) & (t <= d.opto_stimulations.loc[i, 'sample'] + 34)
        axs[row,1].plot(output[s,axis_0], output[s,axis_1])
        
    axs[row,0].set_title(f'visual stimuli period {lineness} manifold')
    axs[row,1].set_title(f'opto stimulation {lineness} manifold')



## Requests from Anne
* train on visual, project everything
* train on everything, project everything
* ignore visual, train and project
for each,
    * eigenspectra
    * scatterplot visual an photostim 


In [ ]:
s = d.neural_data.t < d.opto_stimulations['sample'].min()
first_part_data = NumpyTimedDataSource(d.neural_data.a[s], d.neural_data.t[s])
second_part_data = NumpyTimedDataSource(d.neural_data.a[~s], d.neural_data.t[~s])

In [ ]:
p = Pipeline([
    CenteringTransformer(),
    proSVD(k=10),
])
p.offline_run_on(d.neural_data)

output = p.offline_run_on(d.neural_data, fit=False, convinient_return=False)[0]
output = [x for x in output if not np.isnan(x).any()]
output, t = np.squeeze(output), np.squeeze([x.t for x in output])

In [ ]:
d.sub_dataset

In [ ]:
interesting_combinations = {}
if d.sub_datasets.index(d.sub_dataset) == 2:
    interesting_combinations = {
        "two planes": [
            (1, 3, 6),
            (0, 5, 6),
            (2, 6, 9),
        ],
        "other": [
            (1, 6, 9),
            (0, 6, 9),
            (6, 7, 8),
        ],
        "clam_shapes": [
            (1, 6, 9),
            (0, 3, 6),
            (2, 3, 6),
            (2, 6, 7),
            (0, 4, 6),
            (4, 5, 6),
            (5, 6, 7),
            (3, 5, 6),
            (0, 6, 7),
            (0, 6, 8),
            (0, 2, 6),
            (1, 2, 6),
            (3, 4, 6),
            (6, 8, 9),
        ]
    }
elif d.sub_datasets.index(d.sub_dataset) == 0:
    interesting_combinations = {
        "two planes": [
            (0, 4, 9),
            (0, 5, 8),
            (0, 5, 9), # i guess
        ],
        "plane and a line": [
            (1, 3, 9),
        ],
    }

all_combination_iter = list(itertools.combinations(range(10), 3))
interesting_combinations_iter = [leaf for tree in interesting_combinations.values() for leaf in tree]
combinations = interesting_combinations_iter if True else all_combination_iter
combinations = iter(tqdm.tqdm(combinations))

In [ ]:
%matplotlib qt
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

axis_0, axis_1, axis_2 = next(combinations)


s = t < d.opto_stimulations['sample'].min()
ax.scatter(output[s,axis_0], output[s,axis_1], output[s,axis_2], s=7)
ax.scatter(output[~s,axis_0], output[~s,axis_1], output[~s,axis_2], s=7)

ax.set_xlabel(f'x: {axis_0}')
ax.set_ylabel(f'y: {axis_1}')
ax.set_zlabel(f'z: {axis_2}')
ax.axis('equal');

In [ ]:
def pca(x):
    u, s, vh = np.linalg.svd(x - x.mean(axis=0), full_matrices=False)
    s = s/x.shape[0]
    s_slice = s > 1e-10
    return u[:, s_slice], s[s_slice], vh.T[:, s_slice]

first_part_slice = d.neural_data.t < d.opto_stimulations['sample'].min()

a = np.squeeze(d.neural_data.a)
u0, s0, v0 = pca(a)
u1, s1, v1 = pca(a[first_part_slice])
u2, s2, v2 = pca(a[~first_part_slice])



In [ ]:
%matplotlib inline
fig, ax = plt.subplots()
ax.plot(np.cumsum(s0), label='whole dataset')
ax.plot(np.cumsum(s1), label='visual period')
ax.plot(np.cumsum(s2), label='stim period')
ax.legend()
ax.set_xlabel("# components")
ax.set_ylabel("cumulative variance explained")



In [ ]:
u, s, vh = np.linalg.svd(v1[:,:10].T @ v2[:,:10])
print(s)

i = [0, -1,-2]
recombined_1 = v1[:,:10] @ u[:,i] 
recombined_2 = v2[:,:10] @ vh[i].T
(recombined_1).T @ (recombined_2)

In [ ]:
%matplotlib qt
proj = a @ v0

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(proj[first_part_slice,0], proj[first_part_slice,1], proj[first_part_slice,2], s=7)
ax.scatter(proj[~first_part_slice,0], proj[~first_part_slice,1], proj[~first_part_slice,2], s=7)


In [ ]:
d.sub_dataset

## Neuron locations

In [ ]:
%matplotlib inline
plt.scatter(d.neuron_df['x'], d.neuron_df['y'])


## Video of trajectory

In [ ]:
raise Exception()
s = d.neural_data.t < d.opto_stimulations['sample'].min()
first_part_data = NumpyTimedDataSource(d.neural_data.a[s], d.neural_data.t[s])
p.offline_run_on(first_part_data)

with AnimationManager(n_rows=2, n_cols=2) as am:
    for output in p.run_on(first_part_data):
        am.axs[0,0].scatter(output[0,0], output[0,1])
        
        am.grab_frame()